When model underpredict, if the page rank is high(famous stream), add a huge bias to the predicted value.

When model overpredict, if the page rank is high, minus a huge bias to the predicted value.

The bias is calculated by casting the PR weight it in PR range to the average over/under prediction difference 

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/twitch256/train_streamID2.csv')
#df.columns=['userid','streamid','streamName','time_start','time_end']
#df=df[['userid','streamid','streamName']].iloc[0:200]
df.columns=['userid','streamid','rating']
df.head()

,userid,streamid,rating
0,1,33846768288,2.014098
1,1,33887624992,1.000000
2,1,33890145056,2.014098
3,1,33903958784,2.014098
4,1,33929318864,9.993371


In [ ]:
G = nx.DiGraph() # create direction graph

In [ ]:
#G.add_nodes_from([df['userid'].min(),df['userid'].max()])
G.add_nodes_from([df['userid'].min(),df['userid'].max()])

In [ ]:
for i in list(df['streamid'].unique()): # add nodes : nodes are include both userid and streamid
  G.add_node(i)

In [ ]:
edges=[] # add weighted edges
id=list(df['userid'])
weight = list(df['rating'])
name = list(df['streamid'])
for i in range(df.shape[0]):
  #edges.append((id[i],name[i]))
  edges.append((id[i],name[i],weight[i]))

In [ ]:
#G.add_edges_from(edges)
G.add_weighted_edges_from(edges)

In [ ]:
pr = nx.pagerank(G, alpha=0.85) #create pagerank for all node, but we only take consider the streamID nodes

In [ ]:
over = pd.read_csv('/content/drive/MyDrive/twitch256/overDiff.csv')
under = pd.read_csv('/content/drive/MyDrive/twitch256/underDiff.csv')
over.head(3)

,streamid,filter_diff
0,33863983024,1.852909
1,34164740480,0.547627
2,34180341632,0.820206


In [ ]:
over_pr=[] #store pagerank based on overpredicted streamid
under_pr=[] #store pagerank based on underpredicted streamid
all=[]
for diff in over['streamid']:
  over_pr.append(pr[diff])
for diff in under['streamid']:
  under_pr.append(pr[diff])
for stream in df['streamid']:
  all.append(pr[stream])

In [ ]:
over['pr'] = over_pr #assign to over/under dataframe 
under['pr'] = under_pr
over.head(3)

,streamid,filter_diff,pr
0,33863983024,1.852909,0.000002
1,34164740480,0.547627,0.000002
2,34180341632,0.820206,0.000002


In [ ]:
max(all)/min(all) # just to verify pagerank indeed has a significant range that can used to scale 

113.7692288768512

In [ ]:
overpredict_max_pr=over['pr'].max()
overpredict_min_pr=over['pr'].min()

underpredict_min_pr=under['pr'].min()
underpredict_max_pr=under['pr'].max()

ave_under_diff = 3.2250894353974684 # found from model notebook
ave_over_diff = 1.9021433927814264 # found from model notebook

# use minmax normalization to scale pagerank to the range difference (0-max(overpredict/underpredict difference))
under['under_bias'] = (under['pr']-underpredict_min_pr)/(underpredict_max_pr-underpredict_min_pr)*(ave_under_diff-0)+0 
over['over_bias'] = (over['pr']-overpredict_min_pr)/(overpredict_max_pr-overpredict_min_pr)*(ave_over_diff-0)+0

In [ ]:
over.drop(['filter_diff','pr'],axis=1,inplace=True) # remove unessary columns
under.drop(['filter_diff','pr'],axis=1,inplace=True)

In [ ]:
over.head()

,streamid,over_bias
0,33863983024,0.001062
1,34164740480,0.000665
2,34180341632,0.003111
3,33971009952,0.138117
4,34252848048,0.005494


In [ ]:
over['over_bias'] = over['over_bias']+0.8 # further tune results for better rmse
under['under_bias'] = under['under_bias']+0.1

In [ ]:
testdf = pd.read_csv('/content/drive/MyDrive/twitch256/test_streamID.csv')
testdf.head()

,userid,streamid,rating,pred
0,23122,33863983024,1.000000,3.352909
1,52627,34164740480,3.802557,4.850184
2,20359,34180341632,3.802557,5.122763
3,70716,34181092480,5.344436,3.573955
4,8190,33971009952,1.000000,4.972333


In [ ]:
# merge the overpredict bias file to the MF generate rating file based on same streamID
testdf=testdf.merge(over, on='streamid',how='left')
testdf['over_bias'].fillna(0,inplace=True)
testdf.head()

,userid,streamid,rating,pred,over_bias
0,23122,33863983024,1.000000,3.352909,0.801062
1,52627,34164740480,3.802557,4.850184,0.800665
2,20359,34180341632,3.802557,5.122763,0.803111
3,70716,34181092480,5.344436,3.573955,0.000000
4,8190,33971009952,1.000000,4.972333,0.938117


In [ ]:
# merge the underpredict bias file to the MF generate rating file based on same streamID
testdf=testdf.merge(under, on='streamid',how='left')
testdf['under_bias'].fillna(0,inplace=True)
testdf.head()

,userid,streamid,rating,pred,over_bias,under_bias
0,23122,33863983024,1.000000,3.352909,0.801062,0.00000
1,52627,34164740480,3.802557,4.850184,0.800665,0.00000
2,20359,34180341632,3.802557,5.122763,0.803111,0.00000
3,70716,34181092480,5.344436,3.573955,0.000000,0.11444
4,8190,33971009952,1.000000,4.972333,0.938117,0.00000


In [ ]:
# new rating will be MF_rating - overpredict_bias + underpredict_bias
testdf['pred_with_bias'] = testdf['pred']-testdf['over_bias']+testdf['under_bias']
testdf.head()

,userid,streamid,rating,pred,over_bias,under_bias,pred_with_bias
0,23122,33863983024,1.000000,3.352909,0.801062,0.00000,2.551847
1,52627,34164740480,3.802557,4.850184,0.800665,0.00000,4.049519
2,20359,34180341632,3.802557,5.122763,0.803111,0.00000,4.319652
3,70716,34181092480,5.344436,3.573955,0.000000,0.11444,3.688395
4,8190,33971009952,1.000000,4.972333,0.938117,0.00000,4.034217


In [ ]:
# verify the rmse score
from sklearn.metrics import mean_squared_error

rms = mean_squared_error(testdf['rating'], testdf['pred_with_bias'], squared=False)
print(rms)

2.603690270920952


In [ ]:
# save the result to disk
result_df = testdf[['userid','streamid','pred_with_bias']]
result_df.to_csv('/content/drive/MyDrive/twitch256/submit.csv',index=None)

#save the bias file to disk so it can be used for future new testing data
bias_df = testdf[['streamid','over_bias','under_bias']]
bias_df.to_csv('/content/drive/MyDrive/twitch256/bias.csv',index=None)




---


---



Below is not part of the complete model (testing code only)

In [ ]:
over = pd.read_csv('/content/drive/MyDrive/twitch256/overDiff.csv')
under = pd.read_csv('/content/drive/MyDrive/twitch256/underDiff.csv')

t=over.merge(under,on='streamid',how='inner')
t.shape

(818, 3)

In [ ]:
818/len(testdf['streamid'].unique())

0.005626827171109201

In [ ]:
def tuning(a,b):
  over = pd.read_csv('/content/drive/MyDrive/twitch256/overDiff_full.csv')
  under = pd.read_csv('/content/drive/MyDrive/twitch256/underDiff_full.csv')
  over_pr=[] #store pagerank based on overpredicted streamid
  under_pr=[] #store pagerank based on underpredicted streamid
  all=[]
  for diff in over['streamid']:
    over_pr.append(pr[diff])
  for diff in under['streamid']:
    under_pr.append(pr[diff])
  for stream in df['streamid']:
    all.append(pr[stream])
  over['pr'] = over_pr #assign to over/under dataframe 
  under['pr'] = under_pr

  overpredict_max_pr=over['pr'].max()
  overpredict_min_pr=over['pr'].min()

  underpredict_min_pr=under['pr'].min()
  underpredict_max_pr=under['pr'].max()

  ave_under_diff = 2.2 # found from model notebook
  ave_over_diff = 1.47 # found from model notebook

  under['under_bias'] = (under['pr']-underpredict_min_pr)/(underpredict_max_pr-underpredict_min_pr)*(ave_under_diff-0)+0
  over['over_bias'] = (over['pr']-overpredict_min_pr)/(overpredict_max_pr-overpredict_min_pr)*(ave_over_diff-0)+0

  over['over_bias'] = over['over_bias']+a
  under['under_bias'] = under['under_bias']+b

  testdf = pd.read_csv('/content/drive/MyDrive/twitch256/test_streamID_full.csv')
  testdf=testdf.merge(over, on='streamid',how='left')
  testdf['over_bias'].fillna(0,inplace=True)
  testdf=testdf.merge(under, on='streamid',how='left')
  testdf['under_bias'].fillna(0,inplace=True)
  testdf['pred_with_bias'] = testdf['pred']-testdf['over_bias']+testdf['under_bias']
  rms = mean_squared_error(testdf['rating'], testdf['pred_with_bias'], squared=False)
  return rms
  

In [ ]:
a=0
b=0
result=[]
for i in range(20):
  a+=0.1
  b=0
  for j in range(20):
    b+=0.1
    #print(f'{a},{b}')
    result.append([a,b,tuning(a,b)])

In [ ]:
result.sort(key=lambda k:k[2])